In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn

In [58]:
from functional import make_functional, functional_mod_fw, fw_linearize

In [59]:
net = nn.Sequential(
        nn.Linear(10, 30),
        nn.ReLU(),
        nn.Linear(30, 1)
    )

In [60]:
params, names = make_functional(net)

In [61]:
params = tuple(params)

In [62]:
xb = torch.rand((4, 10))

In [63]:
with torch.no_grad():
    params[0][0, 0] = 0

In [64]:
functional_mod_fw(xb, net, names, *params)

tensor([[-0.2856],
        [-0.1178],
        [-0.1251],
        [-0.1445]], grad_fn=<AddmmBackward>)

In [65]:
# Note: parameters are restored
net

Sequential(
  (0): Linear(in_features=10, out_features=30, bias=True)
  (1): ReLU()
  (2): Linear(in_features=30, out_features=1, bias=True)
)

In [66]:
net_lin, _ = fw_linearize(lambda *params: functional_mod_fw(xb, net, names, *params), \
                          params, create_graph=False, strict=False)

TypeError: cannot assign 'torch.FloatTensor' as parameter 'weight' (torch.nn.Parameter or None expected)

In [69]:
xb.view_as(xb)

tensor([[0.7267, 0.3710, 0.8709, 0.5824, 0.9445, 0.7602, 0.5804, 0.8881, 0.0357,
         0.7174],
        [0.0931, 0.0140, 0.5612, 0.4023, 0.5208, 0.3557, 0.2361, 0.0393, 0.6503,
         0.5295],
        [0.9163, 0.4292, 0.1446, 0.6199, 0.4316, 0.6000, 0.0992, 0.7855, 0.3744,
         0.1023],
        [0.7057, 0.8808, 0.2264, 0.9242, 0.3066, 0.1807, 0.9385, 0.2377, 0.1697,
         0.1883]], grad_fn=<ViewBackward>)